<a href="https://colab.research.google.com/github/gabe-pud/scraping-warframe-wiki-pt-br/blob/main/scraping_warframe_wiki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [ ]:
#corrige url para formato utf-8
import urllib.parse
url = 'https://warframe.fandom.com/pt-br/wiki/Categoria:Armas_Primárias'
url = urllib.parse.urlsplit(url)
url = list(url)
url[2] = urllib.parse.quote(url[2])
url = urllib.parse.urlunsplit(url)
print(url)

https://warframe.fandom.com/pt-br/wiki/Categoria%3AArmas_Prim%C3%A1rias


In [ ]:
#abre a url com BeautifulSoup para receber o codigo fonte
response = urlopen('https://warframe.fandom.com/pt-br/wiki/Categoria:Armas_Prim%C3%A1rias')
html = response.read().decode('utf-8')
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
soup

<!DOCTYPE html>

<html class="client-nojs sse-dtnr-156-control l2u-other" dir="ltr" lang="pt-BR">
<head>
<meta charset="utf-8"/>
<title>Categoria:Armas Primárias | WARFRAME Wiki PT-BR  | Fandom</title>
<script>document.documentElement.className="client-js sse-dtnr-156-control l2u-other";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":[",\t."," \t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","janeiro","fevereiro","março","abril","maio","junho","julho","agosto","setembro","outubro","novembro","dezembro"],"wgRequestId":"f85954ab525621cdcb30e55c78e6bef9","wgCanonicalNamespace":"Category","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":14,"wgPageName":"Categoria:Armas_Primárias","wgTitle":"Armas Primárias","wgCurRevisionId":17180,"wgRevisionId":17180,"wgArticleId":4466,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Armas"],"wgPageViewLanguage":"pt-br","wgPageContentLangua

In [ ]:
#procura todos os links de paginas de armas no codigo fonte da pagina
#que se encontram em <div class="category-page__members">...<a href="pagina da arma">
#e retonra em forma de lista na variavel weapons
weapons = []
for a in soup.find('div',class_='category-page__members').find_all('a', href=True):
    weapons.append(a['href'])

In [ ]:
#remove duplicatas da lista weapons
weapons = list(dict.fromkeys(weapons))

In [ ]:
#remove links que levam para categorias ou armas modulares, fora do escopo
for i in weapons:
  if ('Categoria') in i:
    weapons.remove(i)

weapons.remove('/pt-br/wiki/M%C3%B3dulo:Weapons')

In [ ]:
weapons[4]

'/pt-br/wiki/Artemis_Bow_(Arma)'

In [ ]:
import pandas as pd

In [ ]:
# se a linha de codigo devolve valor nulo devove 0 no lugar
# -------------------
def check_null(line):
  if line is not None:
    return line.div.get_text()
  else:
    return 0

def check_null_ammo(line):
  if line is not None:
    return int(line.div.get_text().split()[0])
  else:
    return 0

def check_null_damage(line):
  if line is not None:
    return float(line.get_text().split()[0])
  else:
    return 0

def check_null_damage_type(line):
  if line is not None:
    return float(line.parent.get_text().split()[0])
  else:
    return 0
# -------------------


#para cada arma em armas entra na pagina e devovle um dicionario com os dados buscados
#depois adiciona o dicionario na lista dataset
# -------------------
dataset = []
for i in range(len(weapons)):
  response = urlopen('https://warframe.fandom.com' + weapons[i])
  html = response.read().decode('utf-8')
  soup = BeautifulSoup(html, 'html.parser')

  row = {
      #nome
      'nome':soup.find('span', style="font-weight:bold;").get_text(),
      #meta
      'slot':soup.find('div', {'data-source':"slot"}).div.get_text(),
      'tipo':soup.find('div', {'data-source':"type"}).div.get_text(),
      'tipo de gatilho':soup.find('div', {'data-source':"trigger"}).div.get_text(),
      'tipo de munição':check_null(soup.find('div', {'data-source':"ammo type"})),
      'taxa de tiro':soup.find('div', {'data-source':"fire rate"}).div.get_text(),
      'capacidade do pente':soup.find('div', {'data-source':"magazine"}).div.get_text(),
      'munição maxima':check_null_ammo(soup.find('div', {'data-source':"max ammo"})),
      'tempo de recarga':soup.find('div', {'data-source':"reload"}).div.get_text(),
      #crit/status
      'chance critica':check_null(soup.find('div', {'data-source':"normal critical chance"})),
      'multiplicador critico':check_null(soup.find('div', {'data-source':"normal critical damage"})),
      'chance de status':check_null(soup.find('div', {'data-source':"normal status chance"})),
      #danos comuns
      'dano colisivo':check_null_damage(soup.find('td', {'data-source':"normal impact"})),
      'dano perfurante':check_null_damage(soup.find('td', {'data-source':"normal puncture"})),
      'dano cortante':check_null_damage(soup.find('td', {'data-source':"normal slash"})),
      #danos elementais
      'dano eletrico':check_null_damage_type(soup.find('span', {'data-param':"Electricity"})),
      'dano de calor':check_null_damage_type(soup.find('span', {'data-param':"Heat"})),
      'dano de frio':check_null_damage_type(soup.find('span', {'data-param':"Cold"})),
      'dano toxico':check_null_damage_type(soup.find('span', {'data-param':"Toxin"})),
      'dano explosivo':check_null_damage_type(soup.find('span', {'data-param':"Blast"})),
      'dano corrosivo':check_null_damage_type(soup.find('span', {'data-param':"Corrosive"})),
      'dano de gas':check_null_damage_type(soup.find('span', {'data-param':"Gas"})),
      'dano magnetico':check_null_damage_type(soup.find('span', {'data-param':"Magnetic"})),
      'dano radioativo':check_null_damage_type(soup.find('span', {'data-param':"Radiation"})),
      'dano viral':check_null_damage_type(soup.find('span', {'data-param':"Viral"}))
      }
  dataset.append(row)
  # -------------------

In [ ]:
response = urlopen('https://warframe.fandom.com/pt-br/wiki/Amprex')
html = response.read().decode('utf-8')
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
def check_null_damage_type(line):
  if line is not None:
    return line.parent.get_text().split()[0]
  else:
    return

In [ ]:
#transforma a lista dataset em um DataFrame(tabela)
df_primarias = pd.DataFrame(dataset)

In [ ]:
df_primarias

,nome,slot,tipo,tipo de gatilho,tipo de munição,taxa de tiro,capacidade do pente,munição maxima,tempo de recarga,chance critica,...,dano eletrico,dano de calor,dano de frio,dano toxico,dano explosivo,dano corrosivo,dano de gas,dano magnetico,dano radioativo,dano viral
0,Acceltra,Primária,Rifle,Auto,Rifle,12.0 tiros por segundo,48 tiros por cartucho,96,2.0 s,32%,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0
1,Amprex,Primária,Rifle,Segurado,Rifle,12.0 tiros por segundo,100 tiros por cartucho,700,2.6 s,32%,...,22.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0
2,Arca Plasmor,Primária,Escopeta,Semi-Auto,Escopeta,1.1 tiros por segundo,10 tiros por cartucho,48,2.8 s,22%,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,600.0,0.0
3,Argonak,Primária,Rifle,Auto / Semi,Rifle,6.0 tiros por segundo,43 tiros por cartucho,473,2.4 s,9%,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0
4,Artemis Bow,Primária,Arma Exalted,Carregado,0,0,1 tiros por cartucho,0,0.9 s,25%,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,Vulkar Wraith,Primária,Sniper Rifle,Semi-Auto,Sniper,2.0 tiros por segundo,8 tiros por cartucho,72,3.0 s,20%,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0
117,Zarr,Primária,Lançadores,Semi-Auto,Sniper,1.67 tiros por segundo,3 tiros por cartucho,84,2.25 s,17%,...,0.0,0.0,0.0,0.0,175.0,0.0,0,0.0,0.0,0.0
118,Zenith,Primária,Rifle,Auto / Semi,Rifle,10.83 tiros por segundo,90 tiros por cartucho,540,1.6 s,10%,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0
119,Zhuge,Primária,Arco,Auto,0,4.17 tiros por segundo,20 tiros por cartucho,540,2.5 s,20%,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0


In [ ]:
teste = df_primarias
df_primarias.sort_values('munição maxima')

,nome,slot,tipo,tipo de gatilho,tipo de munição,taxa de tiro,capacidade do pente,munição maxima,tempo de recarga,chance critica,...,dano eletrico,dano de calor,dano de frio,dano toxico,dano explosivo,dano corrosivo,dano de gas,dano magnetico,dano radioativo,dano viral
4,Artemis Bow,Primária,Arma Exalted,Carregado,0,0,1 tiros por cartucho,0,0.9 s,25%,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0
55,Lenz,Primária,Arco,Carregado,Arco,1.0 tiros por segundo,1 tiros por cartucho,6,0.6 s,0,...,0.0,0.0,10.0,0.0,660.0,0.0,0,0.0,0.0,0.0
63,Ogris,Primária,Lançadores,Carregado,Sniper,1.5 tiros por segundo,5 tiros por cartucho,20,2.5 s,0,...,0.0,0.0,0.0,0.0,100.0,0.0,0,0.0,0.0,0.0
19,Carmine Penta,Primária,Lançador,Ativo,Sniper,2.7 tiros por segundo,10 tiros por cartucho,20,2.5 s,10%,...,0.0,0.0,0.0,0.0,350.0,0.0,0,0.0,0.0,0.0
70,Penta,Primária,Lançadores,Ativo,Sniper,1.0 tiros por segundo,5 tiros por cartucho,20,2.5 s,10%,...,0.0,0.0,0.0,0.0,350.0,0.0,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49,Kohm,Primária,Escopeta,Auto-Spool,Escopeta,3.67 tiros por segundo,245 tiros por cartucho,960,2.0 s,11%,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0
35,Glaxion Vandal,Primária,Rifle,Held,Rifle,12.0 tiros por segundo,100 tiros por cartucho,1000,1.8 s,14%,...,0.0,0.0,29.0,0.0,0.0,0.0,0,0.0,0.0,0.0
74,Prisma Grakata,Primária,Rifle,Auto,Rifle,21.67 tiros por segundo,120 tiros por cartucho,1000,2.0 s,25%,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0
96,Supra,Primária,Rifle,Auto-Spool,Rifle,12.5 tiros por segundo,180 tiros por cartucho,1080,3.0 s,12%,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0


In [ ]:
# arma lançada mais recentemente não encontrado devido ao link inicial usado, que se refere a wiki em portugues, que está desatualizada
df_primarias.query("nome == 'EFV-0 Somnus'")

,nome,slot,tipo,tipo de gatilho,tipo de munição,taxa de tiro,capacidade do pente,munição maxima,tempo de recarga,chance critica,...,dano eletrico,dano de calor,dano de frio,dano toxico,dano explosivo,dano corrosivo,dano de gas,dano magnetico,dano radioativo,dano viral
